In [1]:
import glob
main_path = '/home/diego/Documents/MLCDataset_v1/txts/Label*.txt'

In [2]:
txt_paths = sorted(glob.glob(main_path))

In [3]:
print(txt_paths)

['/home/diego/Documents/MLCDataset_v1/txts/Label_0.txt', '/home/diego/Documents/MLCDataset_v1/txts/Label_1.txt', '/home/diego/Documents/MLCDataset_v1/txts/Label_10.txt', '/home/diego/Documents/MLCDataset_v1/txts/Label_11.txt', '/home/diego/Documents/MLCDataset_v1/txts/Label_12.txt', '/home/diego/Documents/MLCDataset_v1/txts/Label_13.txt', '/home/diego/Documents/MLCDataset_v1/txts/Label_14.txt', '/home/diego/Documents/MLCDataset_v1/txts/Label_2.txt', '/home/diego/Documents/MLCDataset_v1/txts/Label_3.txt', '/home/diego/Documents/MLCDataset_v1/txts/Label_4.txt', '/home/diego/Documents/MLCDataset_v1/txts/Label_5.txt', '/home/diego/Documents/MLCDataset_v1/txts/Label_6.txt', '/home/diego/Documents/MLCDataset_v1/txts/Label_7.txt', '/home/diego/Documents/MLCDataset_v1/txts/Label_8.txt', '/home/diego/Documents/MLCDataset_v1/txts/Label_9.txt']


In [24]:
def join_txt_files(txt_paths, output_path):
    temp_names = []
    with open(output_path, 'w') as outfile:
        for txt_path in txt_paths:
            with open(txt_path) as infile:
                for line in infile:
                    name = line.split('	')[0]
                    if name not in temp_names:
                        temp_names.append(name)
                        outfile.write(line)
    return len(temp_names)

In [25]:
#for i in range(0, len(txt_paths)):
#    print(txt_paths[i])
#    c = join_txt_files([txt_paths[i]], '/home/diego/Documents/MLCDataset_v1/merged.txt')
#    print(c)
#    break
len_names = join_txt_files(txt_paths, '/home/diego/Documents/MLCDataset_v1/merged.txt')
print(len_names)

203


In [30]:
import os
import tqdm

# Function to parse the input text file and convert it to Total Text format
def convert_to_total_text_format(input_file, output_img_dir, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(output_img_dir, exist_ok=True)

    input_dir_path = os.path.dirname(input_file)
    print(input_dir_path)

    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
        for idx, line in enumerate(tqdm.tqdm(lines)):
            try:
                # Split each line into image name and annotations
                parts = line.strip().split('	', 1)
                if len(parts) < 2:
                    continue  # Skip lines that don't have annotations

                image_name = parts[0].split('/')[-1]
                global_image_name = input_dir_path + '/' + image_name
                # copy image to output directory
                new_name = "img{}.jpg".format(idx)
                os.system("cp {} {}".format(global_image_name, output_img_dir + new_name))
                annotations = eval(parts[1].replace("false", "False"))  # Use eval carefully; ensure input is safe

                # Create a filename for the output text file based on the image name
                #filename_without_extension = os.path.splitext(image_name)[0]
                output_file_path = os.path.join(output_dir, new_name.replace('.jpg', '.txt'))

                with open(output_file_path, 'w', encoding='utf-8') as f_out:
                    for annotation in annotations:
                        transcription = annotation["transcription"]
                        if "'" in transcription:
                            transcription = transcription.replace("'", "")
                        if ",":
                            transcription = transcription.replace(",", "") 
                        points = annotation["points"]
                        if "?" in transcription:
                            transcription = transcription.replace("?", "")
                        if "\\" in transcription:
                            transcription = transcription.replace("\\", "")
                        if "/" in transcription:
                            transcription = transcription.replace("/", "")
                        if "[" in transcription:
                            transcription = transcription.replace("[", "")
                        if "]" in transcription:
                            transcription = transcription.replace("]", "")
                        # Extract x and y coordinates
                        x_coords = [int(point[0]) for point in points]
                        y_coords = [int(point[1]) for point in points]

                        # Define orientation (example: 'c' for centered text)
                        orientation = 'c'  # You can change this logic based on your needs

                        # Create the entry for Total Text format
                        entry = (
                            f"x: [{x_coords}], "
                            f"y: [{y_coords}], "
                            f"ornt: [u'{orientation}'], "
                            f"transcriptions: [u'{transcription}']"
                        )

                        # Write each entry to the respective output file
                        f_out.write(f"{entry}\n")
            except:
                continue

# Specify input and output files
input_file_path = '/home/diego/Documents/MLCDataset_v1/merged.txt'  # Replace with your actual input file path
output_img_dir_path = '/home/diego/Documents/MLCDataset_v1/textdet_imgs/test/'  # Directory where images will be saved
output_directory_path = '/home/diego/Documents/MLCDataset_v1/annotations/test/'  # Directory where output files will be saved

# Call the conversion function
convert_to_total_text_format(input_file_path, output_img_dir_path,output_directory_path)

print("Total text format has been written to individual text files in the output directory.")

/home/diego/Documents/MLCDataset_v1


  0%|          | 0/203 [00:00<?, ?it/s]

100%|██████████| 203/203 [00:01<00:00, 110.69it/s]

Total text format has been written to individual text files in the output directory.


In [10]:
!python tools/dataset_converters/prepare_dataset.py paddleann --task textrecog --overwrite-cfg

Dataset Name: Total Text
License Type: BSD-3
License Link: https://github.com/cs-chan/Total-Text-Dataset/blob/master/LICENSE
BibTeX: @article{CK2019, author = {Chee Kheng Chng and Chee Seng Chan and Chenglin Liu}, title = {Total-Text: Towards Orientation Robustness in Scene Text Detection}, journal = {International Journal on Document Analysis and Recognition (IJDAR)}, volume = {23}, pages = {31-52}, year = {2020}, doi = {10.1007/s10032-019-00334-z}}
MMOCR does not own the dataset. Using this dataset you must accept the license provided by the owners, and cite the corresponding papers appropriately.
If you do not agree with the above license, please cancel the progress immediately by pressing ctrl+c. Otherwise, you are deemed to accept the terms and conditions.
5...
4...
3...
2...
1...
Gathering test Dataset...
Parsing test Images and Annotations...
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 203/203, 195.5 task/s, elapsed: 1s, ETA:     0s
Packing test Annotations...
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [39]:
!python tools/visualizations/browse_dataset.py configs/textdet/_base_/datasets/paddleann.py

/home/diego/p_workspace/mo/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '
[>                               ] 10/203, 1.3 task/s, elapsed: 7s, ETA:   144s^C
Traceback (most recent call last):
  File "tools/visualizations/browse_dataset.py", line 415, in <module>
    main()
  File "tools/visualizations/browse_dataset.py", line 381, in main
    image_show = visualizer.add_datasample(
  File "/home/diego/p_workspace/mmocr/mmocr/visualization/textdet_visualizer.py", line 194, in add_datasample
    return self.get_image()
  File "/home/diego/p_workspace/mo/lib/python3.8/site-packages/mmengine/dist/utils.py", line 427, in wrapper
    return func(*args, **kwargs)
  File "/home/diego/p_workspace/mo/lib/python3.8/site-packages/mmengine/visualization/visualizer.py", line 316, in get_image


In [42]:
!CUDA_VISIBLE_DEVICES=0 python tools/test.py configs/textdet/textsnake/textsnake_resnet50-oclip_fpn-unet_1200e_mine.py \
                                             weights/textsnake_resnet50-oclip_fpn-unet_1200e_ctw1500_20221101_134814-a216e5b2.pth

11/08 14:38:15 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.20 (default, Sep  7 2024, 18:35:07) [GCC 13.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 583437386
    GPU 0: NVIDIA GeForce GTX 1650
    CUDA_HOME: /usr
    NVCC: Cuda compilation tools, release 12.0, V12.0.140
    GCC: x86_64-linux-gnu-gcc (Ubuntu 13.2.0-23ubuntu4) 13.2.0
    PyTorch: 1.10.0+cu111
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;

In [2]:
!CUDA_VISIBLE_DEVICES=0 python tools/test.py configs/textdet/fcenet/fcenet_resnet50-oclip_fpn_1500e_ctw1500.py weights/fcenet_resnet50-oclip_fpn_1500e_ctw1500_20221102_121909-101df7e6.pth

11/08 16:45:25 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.20 (default, Sep  7 2024, 18:35:07) [GCC 13.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 648317817
    GPU 0: NVIDIA GeForce GTX 1650
    CUDA_HOME: /usr
    NVCC: Cuda compilation tools, release 12.0, V12.0.140
    GCC: x86_64-linux-gnu-gcc (Ubuntu 13.2.0-23ubuntu4) 13.2.0
    PyTorch: 1.10.0+cu111
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;

In [7]:
!CUDA_VISIBLE_DEVICES=0 python tools/test.py configs/textdet/textsnake/textsnake_resnet50-oclip_fpn-unet_1200e_ctw1500.py \
                                             weights/textsnake_resnet50-oclip_fpn-unet_1200e_ctw1500_20221101_134814-a216e5b2.pth

11/13 15:24:00 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.20 (default, Sep  7 2024, 18:35:07) [GCC 13.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 221297894
    GPU 0: NVIDIA GeForce GTX 1650
    CUDA_HOME: /usr
    NVCC: Cuda compilation tools, release 12.0, V12.0.140
    GCC: x86_64-linux-gnu-gcc (Ubuntu 13.2.0-23ubuntu4) 13.2.0
    PyTorch: 1.10.0+cu111
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;

In [1]:
!CUDA_VISIBLE_DEVICES=0 python tools/test.py configs/textrecog/sar/sar_resnet31_parallel-decoder_5e_st-sub_mj-sub_sa_real.py \
                                             weights/sar_resnet31_parallel-decoder_5e_st-sub_mj-sub_sa_real_20220915_171910-04eb4e75.pth

/home/diego/p_workspace/mo/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
11/15 12:00:16 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.20 (default, Sep  7 2024, 18:35:07) [GCC 13.2.0]
    CUDA available: False
    MUSA available: False
    numpy_random_seed: 727122698
    GCC: x86_64-linux-gnu-gcc (Ubuntu 13.2.0-23ubuntu4) 13.2.0
    PyTorch: 1.10.0+cu111
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel